In [1]:
import numpy as np
import pandas as pd
import glob
import re
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import tqdm

In [2]:
path = "/Users/user/Desktop/Oka/ML-data/24-1-25"
#path ="/Users/aruhy/OneDrive/デスクトップ/data/1-25"

In [4]:
df = pd.DataFrame(columns=['x座標','y座標','z座標','V[V]','T_cell[℃]','W_ch[mm]','i[A/m^2]'])

T_range = ["/*_100℃_1.5ms","/*_110℃_1.5ms","/*_120℃_1.5ms","/*_130℃_1.5ms","/*_140℃_1.5ms","/*_150℃_1.5ms"]
for T in tqdm.tqdm(T_range):
    W_ch_range = ["_0.4mm","_0.8mm","_1.6mm"] #0.2mmはいったん消す
    for W_ch in W_ch_range:
        files2 = glob.glob(path + T + W_ch)
        for filename in tqdm.tqdm(files2):
            
            df1 = pd.read_csv(filename,names=['x座標','y座標','z座標','i[A/m^2]'], delim_whitespace=True, header=8)
            v = re.findall(r'_3D(.*?)V_',filename)
            df1.insert(3,'V[V]', v[0])
            t = re.findall(r'V_(.*?)℃_',filename)
            df1.insert(4,'T_cell[℃]',t[0])
            w = re.findall(r'ms_(.*?)mm',filename)
            df1.insert(5,'W_ch[mm]',w[0])
            df1 = df1.sample(30000)
            df = pd.concat([df, df1])
df['V[V]'] = df['V[V]'].astype(float)
df['T_cell[℃]'] = df['T_cell[℃]'].astype(float)
df['W_ch[mm]'] = df['W_ch[mm]'].astype(float)
df.reset_index(drop=True, inplace=True)
df

  0%|          | 0/6 [00:00<?, ?it/s]C:\Users\user\AppData\Local\Temp\ipykernel_10948\3462138126.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df1])
100%|██████████| 6/6 [01:32<00:00, 15.44s/it]


,x座標,y座標,z座標,V[V],T_cell[℃],W_ch[mm],i[A/m^2]
0,0.000428,0.004925,0.000650,0.45,100.0,0.4,16501.231497
1,0.000212,0.006195,0.000565,0.45,100.0,0.4,16951.397676
2,0.000132,0.007683,0.000588,0.45,100.0,0.4,17055.367421
3,0.000236,0.006862,0.000550,0.45,100.0,0.4,16797.652238
4,0.000324,0.007735,0.000583,0.45,100.0,0.4,16386.853094
...,...,...,...,...,...,...,...
5939995,0.000310,0.003760,0.000286,0.90,150.0,1.6,1899.421782
5939996,0.001450,0.000015,0.000283,0.90,150.0,1.6,1654.887380
5939997,0.001010,0.004242,0.000250,0.90,150.0,1.6,1396.659484
5939998,0.000790,0.000394,0.000250,0.90,150.0,1.6,1486.003894


In [ ]:
def grid_based_sampling(data, grid_count):
    # データの最小値と最大値を計算して、データの範囲を取得
    data_min = np.min(data[:, :3], axis=0)
    data_max = np.max(data[:, :3], axis=0)
    
    # グリッドのサイズを計算
    grid_size = (data_max - data_min) / grid_count
    
   # グリッド内のランダムな代表点を選択してサンプルを構築
    samples = []
    for i in range(grid_count[0]):
        for j in range(grid_count[1]):
            for k in range(grid_count[2]):
                # グリッドの境界を計算
                lower_bound = data_min + np.array([i, j, k]) * grid_size
                upper_bound = lower_bound + grid_size
                
                # グリッド内のデータポイントを抽出
                mask = np.all((data[:, :3] >= lower_bound) & (data[:, :3] < upper_bound), axis=1)
                points_in_grid = data[mask]
                
                if len(points_in_grid) > 0:
                    # グリッド内のデータポイントからランダムに1つ選択して代表点とする
                    selected_point = points_in_grid[np.random.randint(len(points_in_grid))]
                    samples.append(selected_point)
    
    return np.array(samples)

In [ ]:
# 各次元ごとのグリッドの数を指定
grid_count = np.array([3000, 3000, 3000])

In [ ]:
df = pd.DataFrame(columns=['x座標','y座標','z座標','V[V]','T_cell[℃]','W_ch[mm]','i[A/m^2]'])

T_range = ["/*_100℃_1.5ms","/*_110℃_1.5ms","/*_120℃_1.5ms","/*_130℃_1.5ms","/*_140℃_1.5ms","/*_150℃_1.5ms"]
for T in tqdm.tqdm(T_range):
    W_ch_range = ["_0.4mm","_0.8mm","_1.6mm"] #0.2mmはいったん消す
    for W_ch in W_ch_range:
        files2 = glob.glob(path + T + W_ch)
        for filename in tqdm.tqdm(files2):
            
            df1 = pd.read_csv(filename,names=['x座標','y座標','z座標','i[A/m^2]'], delim_whitespace=True, header=8)
            v = re.findall(r'_3D(.*?)V_',filename)
            df1.insert(3,'V[V]', v[0])
            t = re.findall(r'V_(.*?)℃_',filename)
            df1.insert(4,'T_cell[℃]',t[0])
            w = re.findall(r'ms_(.*?)mm',filename)
            df1.insert(5,'W_ch[mm]',w[0])
            df1 = df1.astype(float)
            df1 = pd.DataFrame(grid_based_sampling(df1.values, grid_count))
            df = pd.concat([df, df1])
df['V[V]'] = df['V[V]'].astype(float)
df['T_cell[℃]'] = df['T_cell[℃]'].astype(float)
df['W_ch[mm]'] = df['W_ch[mm]'].astype(float)
df.reset_index(drop=True, inplace=True)
df

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [58:39<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 591000 entries, 0 to 590999
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   x座標        591000 non-null  float64
 1   y座標        591000 non-null  float64
 2   z座標        591000 non-null  float64
 3   V[V]       591000 non-null  float64
 4   T_cell[℃]  591000 non-null  float64
 5   W_ch[mm]   591000 non-null  float64
 6   i[A/m^2]   591000 non-null  float64
dtypes: float64(7)
memory usage: 31.6 MB


In [5]:
df.describe()

,x座標,y座標,z座標,V[V],T_cell[℃],W_ch[mm],i[A/m^2]
count,5.940000e+06,5.940000e+06,5.940000e+06,5.940000e+06,5.940000e+06,5.940000e+06,5.940000e+06
mean,6.665168e-04,5.319164e-03,4.333359e-04,6.500000e-01,1.250000e+02,9.333333e-01,8.386584e+03
std,4.805247e-04,3.109062e-03,1.292532e-04,1.581139e-01,1.707825e+01,4.988877e-01,6.841577e+03
min,4.000000e-06,0.000000e+00,2.500000e-04,4.000000e-01,1.000000e+02,4.000000e-01,-8.682458e-02
25%,2.920000e-04,2.628965e-03,3.251667e-04,5.000000e-01,1.100000e+02,4.000000e-01,2.566528e+03
50%,5.800000e-04,5.319000e-03,3.998333e-04,6.500000e-01,1.250000e+02,8.000000e-01,6.524033e+03
75%,9.300000e-04,8.011005e-03,5.751667e-04,8.000000e-01,1.400000e+02,1.600000e+00,1.297962e+04
max,1.990000e-03,1.063800e-02,6.500000e-04,9.000000e-01,1.500000e+02,1.600000e+00,2.899154e+04


In [6]:
df_re= df.loc[1770000:1799999,'i[A/m^2]']
df_re

1770000   -0.034805
1770001   -0.062490
1770002   -0.042325
1770003   -0.039765
1770004   -0.068109
             ...   
1799995   -0.071232
1799996   -0.065104
1799997   -0.033961
1799998   -0.037627
1799999   -0.035497
Name: i[A/m^2], Length: 30000, dtype: float64

In [7]:
df.drop(df.loc[1770000:1799999,:].index, inplace=True)

In [ ]:
#V=0.65,T=110,W=1.6の値だけ外れていたので補完する
# 条件に基づいて 'i[A/m^2]' のデータを抽出し、その抽出したデータに対して平均を取る
subset1 = df.loc[(df['V[V]'] == 0.6) & (df['T_cell[℃]'] == 110) & (df['W_ch[mm]'] == 1.6), 'i[A/m^2]']
subset2 = df.loc[(df['V[V]'] == 0.7) & (df['T_cell[℃]'] == 110) & (df['W_ch[mm]'] == 1.6), 'i[A/m^2]']

# 平均を計算する
#df_re = (subset1.values + subset2.values) / 2
#df.loc[177000:179999,'i[A/m^2]'] = df_re
#df.loc[177000:179999,'x座標'] = df.loc[174000:176999,'x座標']
#df.loc[177000:179999,'y座標'] = df.loc[174000:176999,'y座標']
#df.loc[177000:179999,'z座標'] = df.loc[174000:176999,'z座標']#ランダムサンプリングされているから座標を合わせるのは無理

In [8]:
df.to_csv("CD3D_100-150℃_0.2-1.6mm_0.4-0.9V")

In [9]:
for col in df.columns:
    scaler =  StandardScaler()
    df[col] = scaler.fit_transform(np.array(df[col].values).reshape(-1, 1))
    
df.head()

,x座標,y座標,z座標,V[V],T_cell[℃],W_ch[mm],i[A/m^2]
0,-0.494016,-0.126724,1.671632,-1.261713,-1.467472,-1.064411,1.181387
1,-0.944598,0.281651,1.012483,-1.261713,-1.467472,-1.064411,1.247271
2,-1.111480,0.760359,1.193072,-1.261713,-1.467472,-1.064411,1.262488
3,-0.894533,0.496452,0.897681,-1.261713,-1.467472,-1.064411,1.224770
4,-0.710963,0.777150,1.155664,-1.261713,-1.467472,-1.064411,1.164647


In [10]:
df.describe()

,x座標,y座標,z座標,V[V],T_cell[℃],W_ch[mm],i[A/m^2]
count,5.910000e+06,5.910000e+06,5.910000e+06,5.910000e+06,5.910000e+06,5.910000e+06,5.910000e+06
mean,-8.172372e-16,1.851326e-15,-2.296051e-16,6.208571e-15,1.862079e-16,-2.557665e-15,-6.238734e-16
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.378491e+00,-1.710802e+00,-1.424173e+00,-1.577141e+00,-1.467472e+00,-1.064411e+00,-1.231137e+00
25%,-7.818880e-01,-8.652211e-01,-8.295205e-01,-9.462846e-01,-8.822656e-01,-1.064411e+00,-8.511636e-01
50%,-1.769406e-01,1.823056e-06,-2.490571e-01,6.303522e-15,2.881473e-01,-2.610050e-01,-2.737228e-01
75%,5.531682e-01,8.658584e-01,1.097618e+00,9.462846e-01,8.733538e-01,1.345807e+00,6.713060e-01
max,2.764355e+00,1.710805e+00,1.671632e+00,1.577141e+00,1.458560e+00,1.345807e+00,3.009404e+00


In [11]:
df.to_csv("CD3D_100-150℃_0.2-1.6mm_0.4-0.9V_std")